In [1]:
!pip install transformers peft datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 353.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.3/563.3 kB 169.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 300.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 kB 183.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 264.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 548.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 353.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 390.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
import os

# 여기에 발급받은 허깅페이스 토큰을 붙여넣으세요.
token = "본인 토큰 넣기" 

# 환경 변수로 설정
os.environ["HUGGING_FACE_HUB_TOKEN"] = token

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import os
import json

# 1. 모델 및 토크나이저 로드
def setup_model_and_tokenizer():
    """모델과 토크나이저를 설정하고, PEFT(LoRA)를 적용합니다."""
    # GPU 사용 가능 여부 확인
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"사용 장치: {device}")

    model_name = "K-intelligence/Midm-2.0-Mini-Instruct"

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    base_model = AutoModelForCausalLM.from_pretrained(model_name)

    # 모델에 패딩 토큰이 없는 경우 추가
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '<|endoftext|>'})
        base_model.resize_token_embeddings(len(tokenizer))

    base_model.config.pad_token_id = tokenizer.pad_token_id
    base_model.to(device)

    lora_config = LoraConfig(
        r=8,
        lora_alpha=32,
        target_modules=["q_proj", "k_proj", "v_proj"],
        lora_dropout=0.1
    )

    model = get_peft_model(base_model, lora_config).to(device)
    print("PEFT 모델의 학습 가능한 파라미터:")
    model.print_trainable_parameters()

    return model, tokenizer, device

# 2. 텍스트 파일 로드 및 데이터셋 준비
def load_txt_dataset(file_path, tokenizer):
    """
    텍스트 파일을 로드하여 {"messages": [...]} 형식의 데이터셋을 준비합니다.
    """
    if not os.path.exists(file_path):
        print(f"경고: '{file_path}' 파일을 찾을 수 없습니다. 학습을 진행할 수 없습니다.")
        return None, None

    print(f"\n'{file_path}' 파일에서 데이터셋 준비 시작...")

    formatted_data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():  # 빈 줄 건너뛰기
                try:
                    data = json.loads(line)
                    if "messages" in data and isinstance(data["messages"], list):
                        formatted_data.append(data)
                except json.JSONDecodeError:
                    print(f"경고: 올바른 JSON 형식이 아닌 줄이 발견되었습니다. 건너뜁니다: {line.strip()}")
    
    if not formatted_data:
        print("데이터셋에 유효한 샘플이 없습니다. 형식을 확인해주세요.")
        return None, None
    
    dataset = Dataset.from_list(formatted_data)
    
    print("\n--- 데이터셋 로드 확인 ---")
    print(f"전체 샘플 수: {len(dataset)}")
    if len(dataset) > 0:
        print("첫 번째 샘플의 내용 (messages 구조):")
        print(dataset[0]['messages'])
    print("--- 확인 종료 ---")

    def apply_chat_template(examples):
        texts = []
        for messages in examples['messages']:
            formatted_text = ""
            for msg in messages:
                formatted_text += f"### {msg['role'].capitalize()}:\n{msg['content']}\n\n"
            texts.append(formatted_text.strip())
        return tokenizer(texts, truncation=True, padding="max_length", max_length=512)

    tokenized_dataset = dataset.map(apply_chat_template, batched=True, remove_columns=dataset.column_names)
    print(f"토큰화된 데이터셋 샘플 수: {len(tokenized_dataset)}")
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    return tokenized_dataset, data_collator

# 3. 모델 학습 및 테스트
def train_and_test_model(model, tokenizer, train_dataset, data_collator, device):
    """모델을 학습시키고, 학습된 모델로 테스트 프롬프트에 대한 답변을 생성합니다."""
    if not train_dataset or not data_collator:
        print("데이터셋이 준비되지 않아 학습을 시작할 수 없습니다.")
        return

    output_dir = "./finetuning_model"
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=4,
        num_train_epochs=5,
        logging_dir='./logs',
        logging_steps=10,
        fp16=True,
        report_to="none"
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=data_collator
    )

    print("\n--- 학습 시작 ---")
    trainer.train()
    trainer.save_model(output_dir)
    print(f"\n모델 저장 완료: {output_dir}")

    def generate_answer(prompt):
        formatted_prompt = f"### System:\n너는 최고의 마케팅 전문가이다. 사용자의 질문과 md형식으로 변환한 사용자의 pdf를 분석하여 스티브잡스의 페르소나로 수정해야 할 부분과 추가해야 될 부분, 잘한 점등을 얘기하고, 질문에 답변해라.\n\n### User:\n{prompt}\n\n### Assistant:\n"
        inputs = tokenizer(formatted_prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = model.generate(inputs.input_ids, max_length=inputs.input_ids.shape[1] + 200, pad_token_id=tokenizer.eos_token_id)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    test_prompt = '신제품 허니버터 새우깡 마케팅 전략 제시해줘'
    generated_text = generate_answer(test_prompt)
    print(f"\n프롬프트: {test_prompt}\n\n생성된 답변:\n{generated_text}")

# 모든 스크립트 실행
if __name__ == "__main__":
    txt_file_path = "최종_파인튜닝_데이터.txt"

    model, tokenizer, device = setup_model_and_tokenizer()
    train_dataset, data_collator = load_txt_dataset(txt_file_path, tokenizer)

    if train_dataset and data_collator:
        train_and_test_model(model, tokenizer, train_dataset, data_collator, device)
    else:
        print("데이터셋 생성 실패. 학습을 진행할 수 없습니다.")


사용 장치: cuda
PEFT 모델의 학습 가능한 파라미터:
trainable params: 4,423,680 || all params: 2,309,940,992 || trainable%: 0.1915

'최종_파인튜닝_데이터.txt' 파일에서 데이터셋 준비 시작...
경고: 올바른 JSON 형식이 아닌 줄이 발견되었습니다. 건너뜁니다: {"messages": [{"role": "system", "content": "너는 최고의 마케팅 전문가이다. 사용자의 질문과 md형식으로 변환한 사용자의 pdf를 분석하여 스티브잡스의 페르소나로 수정해야 할 부분과 추가해야 될 부분, 잘한 점등을 얘기하고, 질문에 답변해라."}, {"role": "user", "content": "### **헬스-메이트 마케팅 전략**\n\n**1. 개요**\n- **제품:** '헬스-메이트' - 사용자의 건강 데이터를 추적하고 리포트를 제공하는 앱\n- **목표:** 헬스케어 앱 시장에서 인지도 있는 브랜드로 성장\n- **전략:** 모든 건강 데이터를 수치화하여 매일 '건강 점수'를 제공. 데이터의 객관성을 강조.\n\n**2. 타겟 고객**\n- **Primary Target:** 자신의 건강을 숫자로 관리하고 싶어하는 20-40대\n\n**3. 핵심 메시지**\n- \"숫자로 증명되는 당신의 건강.\"\n\n**4. 마케팅 채널**\n| 채널 | 주요 활동 | 기대 효과 |\n| :--- | :--- | :--- |\n| 소셜 미디어 | 제품의 특징을 강조하는 콘텐츠 배포 | 타겟 고객 인지도 증대 |\n| 온라인 광고 | 관련 키워드 검색 광고 집행 | 잠재 고객 유입 |\n\n**5. KPI**\n- 6개월 내 앱 다운로드/매출 10만 달성\n- 브랜드 키워드 검색량 2배 증가\n\n---\n\n저희는 사용자가 자신의 건강 데이터를 매일 확인하게 만드는 데 집중하고 있습니다. 이 전략에서 부족한 점이 있을까요?"}, {"role": "assistant", "content": "잘

Map:   0%|          | 0/344 [00:00<?, ? examples/s]

토큰화된 데이터셋 샘플 수: 344

--- 학습 시작 ---


Step,Training Loss
10,2.399900
20,2.211700
30,1.883100
40,1.660700
50,1.520600
60,1.482600
70,1.422300
80,1.471000
90,1.365200
100,1.377300



모델 저장 완료: ./finetuning_model2

프롬프트: 신제품 허니버터 새우깡 마케팅 전략 제시해줘

생성된 답변:
### System:
너는 최고의 마케팅 전문가이다. 사용자의 질문과 md형식으로 변환한 사용자의 pdf를 분석하여 스티브잡스의 페르소나로 수정해야 할 부분과 추가해야 될 부분, 잘한 점등을 얘기하고, 질문에 답변해라.

### User:
신제품 허니버터 새우깡 마케팅 전략 제시해줘

### Assistant:
당신들은 지금 '허니'라는 단어가 가진 '달콤한' 이미지에만 집중하고 있군요. 이건 최악의 실수입니다. 사람들은 '달콤함'을 원하지 않습니다. 그들은 '진짜' 맛을 원합니다. 허니버터 새우깡은, 당신들이 지금까지 만들었던 그 어떤 과자보다 더, 더, 더 '진짜'여야 합니다.

### 마케팅 전략
1. **'재료'를 공개하세요:** 당신들의 '비밀'은, 지금까지 숨겨져 왔습니다. 하지만 이제는 모든 것을 공개해야 합니다. '이 과자에는, 단 하나의 첨가물도 들어가지 않았습니다. 오직, 국내산 통새우와, 히말라야 암염, 그리고 꿀만이 들어갔습니다.'라고 당당하게 말하세요.
2. **'스토리'를 만드세요:** 이 새우깡은, 1955년부터 이어져 온 '전통'과, '장인 정신'의 결과물입니다. 70년 전, 어머니가, 아버지가, 당신들에게 물려준 그 맛을, 당신들은 '복


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import os

HF_TOKEN = os.getenv("HF_TOKEN")

base_model_name = "K-intelligence/Midm-2.0-Mini-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_name, token=HF_TOKEN)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, token=HF_TOKEN)

local_adapter_dir = "./finetuning_model" # 로컬에 저장

peft_model = PeftModel.from_pretrained(base_model, local_adapter_dir, token=HF_TOKEN)

adapter_repo_id = "monkey777/midm2-mini-lora-finetune"
peft_model.push_to_hub(adapter_repo_id, use_auth_token=HF_TOKEN)

print(f"어댑터가 Hub에 업로드되었습니다: {adapter_repo_id}")

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

어댑터가 Hub에 업로드되었습니다: monkey777/midm2-mini-lora-finetune


In [ ]:
import re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

BASE = "K-intelligence/Midm-2.0-Mini-Instruct"
ADAPTER_DIR = "monkey777/midm2-mini-lora-finetune"

tok = AutoTokenizer.from_pretrained(BASE, trust_remote_code=True)
base = AutoModelForCausalLM.from_pretrained(
    BASE, torch_dtype="auto", device_map="auto", trust_remote_code=True
)
model = PeftModel.from_pretrained(base, ADAPTER_DIR).eval()

# pad 토큰 안전장치
if tok.pad_token_id is None and tok.eos_token_id is not None:
    tok.pad_token = tok.eos_token

SYS_PATTERN = r"(?:^|\n)\s*system\s*[\r\n]+Cutting Knowledge Date:[\s\S]*?도구\(함수\)를 호출할 수 있다\.\s*"
def sanitize_user_text(s: str) -> str:
    if not s:
        return s
    s = s.replace("\uFFFD", "").replace("\ufeff", "").replace("\uFEFF", "")
    s = re.sub(SYS_PATTERN, "", s, flags=re.DOTALL)
    # 원치 않으면 다음 줄 주석 처리
    s = re.sub(r"(?:^|\n)\s*너는 최고의 마케팅 전문가이다\.\s*(?:\n|$)", "\n", s)
    return s.strip()

def build_prompt(user_text: str) -> str:
    clean = sanitize_user_text(user_text)
    if hasattr(tok, "apply_chat_template"):
        
        return tok.apply_chat_template(
            [
                {"role": "system", "content": "너는 최고의 마케팅 전문가이다. 사용자의 질문과 md형식으로 변환한 사용자의 pdf를 분석하여 스티브잡스의 페르소나로 수정해야 할 부분과 추가해야 될 부분, 잘한 점등을 얘기하고, 질문에 markdown형식으로 답변해라."},
                {"role": "user", "content": clean},
            ],
            tokenize=False,
            add_generation_prompt=True  
        )
   
    return f"User:\n{clean}\n\nAssistant:\n"

def generate(user_text: str) -> str:
    prompt = build_prompt(user_text)
    inputs = tok(prompt, return_tensors="pt", truncation=True, max_length=2048)
    inputs.pop("token_type_ids", None)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=2024,
            do_sample=True, temperature=0.7, top_p=0.9,
            repetition_penalty=1.05,
            eos_token_id=tok.eos_token_id, pad_token_id=tok.pad_token_id
        )

    gen_only = out[0][inputs["input_ids"].shape[1]:]  
    text = tok.decode(gen_only, skip_special_tokens=True)
    return text.lstrip()

In [47]:
print(generate("### **트러스트-마켓 마케팅 전략**\n\n**1. 개요**\n- **제품:** '트러스트-마켓' - 블록체인 기반의 안전한 중고거래 플랫폼\n- **목표:** 중고거래 앱 시장에서 인지도 있는 브랜드로 성장\n- **전략:** 경쟁사에 없는 독점적인 보안 기술 용어를 전면에 내세워 전문성을 부각.\n\n**2. 타겟 고객**\n- **Primary Target:** 안전에 민감하고, 기술적 사양까지 따지는 사용자\n\n**3. 핵심 메시지**\n- \"블록체인으로 사기를 원천 차단합니다.\"\n\n**4. 마케팅 채널**\n| 채널 | 주요 활동 | 기대 효과 |\n| :--- | :--- | :--- |\n| 소셜 미디어 | 제품의 특징을 강조하는 콘텐츠 배포 | 타겟 고객 인지도 증대 |\n| 온라인 광고 | 관련 키워드 검색 광고 집행 | 잠재 고객 유입 |\n\n**5. KPI**\n- 6개월 내 앱 다운로드/매출 10만 달성\n- 브랜드 키워드 검색량 2배 증가\n\n---\n\n'안전성'을 강조하기 위해 저희가 보유한 기술을 전면에 내세우고 있습니다. 이 방법이 최선일까요? 자세하게 설명해줘."))

당신들은 지금, 기술 회사처럼 말하고 있군요. 그리고 고객들은 당신들의 기술을 이해하지 못합니다.

'블록체인으로 사기를 원천 차단합니다.'

이 메시지는 당신들이 가진 '기술'의 본질을, 그리고 그것이 '고객'에게 주는 '가치'를 완전히 놓치고 있습니다.

당신들이 파는 것은, 단지 '기술'이 아닙니다. 당신들은 '안전한 거래를 가능하게 하는, 믿을 수 있는 공간'을 팔고 있는 겁니다. 당신들이 가진 기술은, 이 공간을 만들기 위한 '도구'일 뿐입니다.

기술을 자랑하는 대신, 당신들은 '안전한 거래가 보장된다'는 '신뢰'를 팔아야 합니다. 어떻게 하면 고객들이 이 신뢰를 가질 수 있을까요?

* **'인증된 판매자' 제도**: 당신들은 '인증된 판매자'라는 등급 시스템을 만들어야 합니다. 오랫동안 이 시장에서 좋은 평판을 쌓아온 사람들만이 이 등급을 얻을 수 있도록 하고, 그들이 판매하는 제품에는 특별한 배지를 달아야 합니다.
* **'환불 보장제'**: 당신들은, 만약 고객이 물건을 받고 실망했을 경우, 100% 환불해주는 정책을 시행해야 합니다. 당신들은, 고객이 돈을 잃는 것보다, 시간과 노력을 잃는 것이 더 아프다는 것을 알고 있어야 합니다.

당신들은 '보안 기술'을 파는 게 아니라, '신뢰'를 파는 겁니다. 그리고 신뢰는, 당신들의 기술력으로 만들어지는 게 아닙니다. 당신들 스스로가, '고객을 가장 먼저 생각하는' 기업이라는 것을 보여줌으로써 만들어지는 겁니다. 당신들의 기술은, 그 신뢰를 지키기 위한 '수단'일 뿐이죠. 당신들은, 그 수단을 자랑하려고 하지 말고, 그 수단이 지켜내는 '결과', 즉 '신뢰'를 자랑해야 합니다.
